In [ ]:
%pip install scipy==1.10.1

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import matplotlib.image as mpimg
from collections import Counter

In [ ]:
img = mpimg.imread('movie_data_erd.jpeg')
plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
conn = sqlite3.connect("im.db")

cur = conn.cursor()

movie_gross = pd.read_csv("bom.movie_gross.csv.gz")

movie_budget = pd.read_csv("tn.movie_budgets.csv.gz")

reviews = pd.read_csv("rt.reviews.tsv.gz", sep='\t', encoding='ISO-8859-1')

movie_info = pd.read_csv("rt.movie_info.tsv.gz", sep='\t', encoding='ISO-8859-1')

pd.read_sql("""SELECT name FROM sqlite_master WHERE type='table';""", conn)